In [ ]:
# Copyright (c) 2024 Microsoft Corporation.
# Licensed under the MIT License.

In [1]:
import os

import pandas as pd
import tiktoken

from graphrag.query.context_builder.entity_extraction import EntityVectorStoreKey
from graphrag.query.indexer_adapters import (
    read_indexer_covariates,
    read_indexer_entities,
    read_indexer_relationships,
    read_indexer_reports,
    read_indexer_text_units,
)
from graphrag.query.question_gen.local_gen import LocalQuestionGen
from graphrag.query.structured_search.local_search.mixed_context import (
    LocalSearchMixedContext,
)
from graphrag.query.structured_search.local_search.search import LocalSearch
from graphrag.vector_stores.lancedb import LanceDBVectorStore

import networkx as nx
from tqdm import tqdm

## Local Search Example

Local search method generates answers by combining relevant data from the AI-extracted knowledge-graph with text chunks of the raw documents. This method is suitable for questions that require an understanding of specific entities mentioned in the documents (e.g. What are the healing properties of chamomile?).

### Load text units and graph data tables as context for local search

- In this test we first load indexing outputs from parquet files to dataframes, then convert these dataframes into collections of data objects aligning with the knowledge model.

### Load tables to dataframes

In [2]:
#MODEL = "deepseek-chat"
#MODEL = "llama-3.3-70b-versatile"
MODEL = "gpt-4o-mini"
INPUT_DIR = f"./output/{MODEL}"
LANCEDB_URI = f"{INPUT_DIR}/lancedb"

COMMUNITY_REPORT_TABLE = "community_reports"
ENTITY_TABLE = "entities"
COMMUNITY_TABLE = "communities"
RELATIONSHIP_TABLE = "relationships"
COVARIATE_TABLE = "covariates"
TEXT_UNIT_TABLE = "text_units"
COMMUNITY_LEVEL = 2

#### Read entities

In [3]:
# read nodes table to get community and degree data
entity_df = pd.read_parquet(f"{INPUT_DIR}/{ENTITY_TABLE}.parquet")
community_df = pd.read_parquet(f"{INPUT_DIR}/{COMMUNITY_TABLE}.parquet")

entities = read_indexer_entities(entity_df, community_df, COMMUNITY_LEVEL)

# load description embeddings to an in-memory lancedb vectorstore
# to connect to a remote db, specify url and port values.
description_embedding_store = LanceDBVectorStore(
    collection_name="default-entity-description",
)
description_embedding_store.connect(db_uri=LANCEDB_URI)

print(f"Entity count: {len(entity_df)}")
entity_df.head()

Entity count: 2699


,id,human_readable_id,title,type,description,text_unit_ids,frequency,degree,x,y
0,04b4cc08-c068-41fe-a736-7d80bfdafdbc,0,TENEBRIO MOLITOR,ORGANISM,"Tenebrio molitor, commonly known as the yellow...",[a88ced7dae53e47df6db4c1f5140556bea6efd47ab8e9...,481,521,0.0,0.0
1,c89fbdf6-a812-48ba-9786-f494d821d4a9,1,POLYSTYRENE,CHEMICAL,Polystyrene (PS) is a widely utilized syntheti...,[a88ced7dae53e47df6db4c1f5140556bea6efd47ab8e9...,492,364,0.0,0.0
2,27154273-c405-4a28-bde2-aba12441b988,2,BIODEGRADATION,BIODEGRADATION,Biodegradation is a critical natural process i...,[a88ced7dae53e47df6db4c1f5140556bea6efd47ab8e9...,566,565,0.0,0.0
3,928a2ff6-20ab-4460-b9bb-1594f50fc785,3,MICROPLASTICS,MICROPLASTIC,Microplastics (MPs) are tiny plastic particles...,[a88ced7dae53e47df6db4c1f5140556bea6efd47ab8e9...,116,67,0.0,0.0
4,1558c835-20c6-47cc-9224-3f8e16cc4c32,4,ENVIRONMENTAL POLLUTION,ENVIRONMENTAL POLLUTANT,Environmental pollution refers to the introduc...,[a88ced7dae53e47df6db4c1f5140556bea6efd47ab8e9...,45,22,0.0,0.0


#### Read relationships

In [4]:
relationship_df = pd.read_parquet(f"{INPUT_DIR}/{RELATIONSHIP_TABLE}.parquet")
relationships = read_indexer_relationships(relationship_df)

print(f"Relationship count: {len(relationship_df)}")
relationship_df.head()

Relationship count: 3748


,id,human_readable_id,source,target,description,weight,combined_degree,text_unit_ids
0,4ada600d-7514-4a79-bb5f-ddea242c2c2e,0,TENEBRIO MOLITOR,POLYSTYRENE,"Tenebrio molitor, commonly known as mealworm l...",2081.0,885,[a88ced7dae53e47df6db4c1f5140556bea6efd47ab8e9...
1,6f515c54-945a-4e69-9b85-f3a9e084a4d3,1,POLYSTYRENE,BIODEGRADATION,Polystyrene (PS) is a polymer recognized for i...,1039.0,929,[a88ced7dae53e47df6db4c1f5140556bea6efd47ab8e9...
2,8274c26d-bdae-42d3-842a-377938539145,2,BIODEGRADATION,SUPPLEMENTAL NUTRITION,Supplemental nutrition enhances the rates of b...,7.0,566,[a88ced7dae53e47df6db4c1f5140556bea6efd47ab8e9...
3,258aae4b-bfb6-4efd-895f-b793f8f4bd1e,3,BIODEGRADATION,LIFE CYCLE,"The study of biodegradation, particularly in t...",9.0,567,[a88ced7dae53e47df6db4c1f5140556bea6efd47ab8e9...
4,fb5c8a73-3b93-4cdc-9ada-a96833d4dc38,4,MICROPLASTICS,ENVIRONMENTAL POLLUTION,Microplastics are a significant and pervasive ...,67.0,89,[a88ced7dae53e47df6db4c1f5140556bea6efd47ab8e9...


In [5]:
%%script false --no-raise-error
# NOTE: covariates are turned off by default, because they generally need prompt tuning to be valuable
# Please see the GRAPHRAG_CLAIM_* settings
covariate_df = pd.read_parquet(f"{INPUT_DIR}/{COVARIATE_TABLE}.parquet")

claims = read_indexer_covariates(covariate_df)

print(f"Claim records: {len(claims)}")
covariates = {"claims": claims}

Couldn't find program: 'false'


#### Read community reports

In [6]:
report_df = pd.read_parquet(f"{INPUT_DIR}/{COMMUNITY_REPORT_TABLE}.parquet")
reports = read_indexer_reports(report_df, community_df, COMMUNITY_LEVEL)

print(f"Report records: {len(report_df)}")
report_df.head()

Report records: 221


,id,human_readable_id,community,level,parent,children,title,summary,full_content,rank,rating_explanation,findings,full_content_json,period,size
0,e199ef0aa7934ce19073a7572620eac2,182,182,2,20,[],Mealworms and Plastic Biodegradation Research ...,This community focuses on the biodegradation o...,# Mealworms and Plastic Biodegradation Researc...,9.5,The community's research on mealworms' biodegr...,"[{'explanation': 'Frass, the excrement of Tene...","{\n ""title"": ""Mealworms and Plastic Biodegr...",2025-03-20,28
1,2cefccd402ca4bc08e650091ab3a4988,183,183,2,20,[],Plastic Biodegradation Community: Mealworms an...,This community focuses on the research surroun...,# Plastic Biodegradation Community: Mealworms ...,9.5,The impact severity rating is high due to the ...,[{'explanation': 'Gas Chromatography-Mass Spec...,"{\n ""title"": ""Plastic Biodegradation Commun...",2025-03-20,2
2,fbf63ab9284b4acea7abb82ecbaa1a36,184,184,2,20,[],Mealworm Biodegradation Research Community,The community focuses on the interactions betw...,# Mealworm Biodegradation Research Community\n...,8.5,The rating reflects the significant relevance ...,[{'explanation': 'Ion chromatography is a cruc...,"{\n ""title"": ""Mealworm Biodegradation Resea...",2025-03-20,2
3,36f60c36a0114702be14584f29c6645f,185,185,2,22,[],Mealworms and PVC Biodegradation Community,The community focuses on the biodegradation of...,# Mealworms and PVC Biodegradation Community\n...,9.0,The rating reflects the significant potential ...,[{'explanation': 'Research indicates that Tene...,"{\n ""title"": ""Mealworms and PVC Biodegradat...",2025-03-20,5
4,a8af729af2714a31815be93c83aeb403,186,186,2,22,[],Mealworms and Plastic Biodegradation Research ...,This community centers around the research on ...,# Mealworms and Plastic Biodegradation Researc...,9.0,The impact severity rating is high due to the ...,[{'explanation': 'Depolymerization is essentia...,"{\n ""title"": ""Mealworms and Plastic Biodegr...",2025-03-20,3


#### Read text units

In [7]:
text_unit_df = pd.read_parquet(f"{INPUT_DIR}/{TEXT_UNIT_TABLE}.parquet")
text_units = read_indexer_text_units(text_unit_df)

print(f"Text unit records: {len(text_unit_df)}")
text_unit_df.head()

Text unit records: 901


,id,human_readable_id,text,n_tokens,document_ids,entity_ids,relationship_ids,covariate_ids
0,a88ced7dae53e47df6db4c1f5140556bea6efd47ab8e95...,1,# Biodegradation of polystyrene wastes in yell...,1200,[02266e3ecb4f5e768940af76832fcc8db8311351619c2...,"[04b4cc08-c068-41fe-a736-7d80bfdafdbc, c89fbdf...","[4ada600d-7514-4a79-bb5f-ddea242c2c2e, 6f515c5...",[]
1,af21c8a728028c41e0853835613161369e766630831ab8...,2,sold as fertilizer. Researchers have conclude...,1200,[02266e3ecb4f5e768940af76832fcc8db8311351619c2...,"[04b4cc08-c068-41fe-a736-7d80bfdafdbc, 2715427...","[258aae4b-bfb6-4efd-895f-b793f8f4bd1e, 1a64c0e...",[]
2,d46f7ff12d14a5779807bbab3156f988bd3dbdb7946c24...,3,B:PS ratio) was 8:1 g/g over a 32-day period....,1200,[02266e3ecb4f5e768940af76832fcc8db8311351619c2...,"[c89fbdf6-a812-48ba-9786-f494d821d4a9, 150ef47...","[efdfcc5f-d43c-4ec5-a958-780d9b319ea5, 55fc0cc...",[]
3,5596ac3f0595b12b50cc4f463539c5678acfe0eda3a703...,4,T. molitor strain CA mealworms fed PS alone. ...,1200,[02266e3ecb4f5e768940af76832fcc8db8311351619c2...,"[04b4cc08-c068-41fe-a736-7d80bfdafdbc, c89fbdf...","[4ada600d-7514-4a79-bb5f-ddea242c2c2e, 6f515c5...",[]
4,a942bde604492e0f880583baef33b2a608d0e57d2b773f...,5,) decreased from 66.2 ± 2.3% on day 4 to 35.2 ...,1200,[02266e3ecb4f5e768940af76832fcc8db8311351619c2...,"[c89fbdf6-a812-48ba-9786-f494d821d4a9, 2715427...","[6f515c54-945a-4e69-9b85-f3a9e084a4d3, e958317...",[]


In [8]:
from graphrag.config.enums import ModelType
from graphrag.config.models.language_model_config import LanguageModelConfig
from graphrag.language_model.manager import ModelManager
from dotenv import load_dotenv
load_dotenv('.env')

qwen_api_key = os.environ["QWEN_API_KEY"]
groq_api_key = os.environ["GROQQ_API_KEY"]
ds_api_key = os.environ["DEEPSEEK_API_KEY"]
api_key = os.environ["GRAPHRAG_API_KEY"]
embedding_model = os.environ["GRAPHRAG_EMBEDDING_MODEL"]
llm_model = MODEL

chat_config = LanguageModelConfig(
    api_key=api_key,
    api_base="https://api.openai.com/v1",
    type=ModelType.OpenAIChat,
    model=MODEL,
    max_retries=2,
    encoding_model="cl100k_base"
)

chat_model = ModelManager().get_or_create_chat_model(
    name="local_search",
    model_type=ModelType.OpenAIChat,
    config=chat_config,
)

token_encoder = tiktoken.get_encoding("cl100k_base")

embedding_config = LanguageModelConfig(
    api_key=api_key,
    type=ModelType.OpenAIEmbedding,
    model=embedding_model,
    max_retries=2,
)

text_embedder = ModelManager().get_or_create_embedding_model(
    name="local_search_embedding",
    model_type=ModelType.OpenAIEmbedding,
    config=embedding_config,
)

### Create local search context builder

In [9]:
context_builder = LocalSearchMixedContext(
    community_reports=reports,
    text_units=text_units,
    entities=entities,
    relationships=relationships,
    # if you did not run covariates during indexing, set this to None
    covariates=None,
    entity_text_embeddings=description_embedding_store,
    embedding_vectorstore_key=EntityVectorStoreKey.ID,  # if the vectorstore uses entity title as ids, set this to EntityVectorStoreKey.TITLE
    text_embedder=text_embedder,
    token_encoder=token_encoder,
)

### Create local search engine

In [10]:
# text_unit_prop: proportion of context window dedicated to related text units
# community_prop: proportion of context window dedicated to community reports.
# The remaining proportion is dedicated to entities and relationships. Sum of text_unit_prop and community_prop should be <= 1
# conversation_history_max_turns: maximum number of turns to include in the conversation history.
# conversation_history_user_turns_only: if True, only include user queries in the conversation history.
# top_k_mapped_entities: number of related entities to retrieve from the entity description embedding store.
# top_k_relationships: control the number of out-of-network relationships to pull into the context window.
# include_entity_rank: if True, include the entity rank in the entity table in the context window. Default entity rank = node degree.
# include_relationship_weight: if True, include the relationship weight in the context window.
# include_community_rank: if True, include the community rank in the context window.
# return_candidate_context: if True, return a set of dataframes containing all candidate entity/relationship/covariate records that
# could be relevant. Note that not all of these records will be included in the context window. The "in_context" column in these
# dataframes indicates whether the record is included in the context window.
# max_tokens: maximum number of tokens to use for the context window.


local_context_params = {
    "text_unit_prop": 0.5,
    "community_prop": 0.1,
    "conversation_history_max_turns": 5,
    "conversation_history_user_turns_only": True,
    "top_k_mapped_entities": 10,
    "top_k_relationships": 10,
    "include_entity_rank": True,
    "include_relationship_weight": True,
    "include_community_rank": False,
    "return_candidate_context": False,
    "embedding_vectorstore_key": EntityVectorStoreKey.ID,  # set this to EntityVectorStoreKey.TITLE if the vectorstore uses entity title as ids
    "max_tokens": 12_000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 5000)
}

model_params = {
    "max_tokens": 2_000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 1000=1500)
    "temperature": 0.6,
}

In [11]:
search_engine = LocalSearch(
    model=chat_model,
    context_builder=context_builder,
    token_encoder=token_encoder,
    model_params=model_params,
    context_builder_params=local_context_params,
    response_type="multiple paragraphs",  # free form text describing the response type and format, can be anything, e.g. prioritized list, single paragraph, multiple paragraphs, multiple-page report
)

### Graph Visualization

In [12]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as mcolors
selected_node_columns = ["community", "degree"]
selected_edge_columns = ["weight", "description", "combined_degree"]
# converts the entities dataframe to a list of dicts for NetworkX
def convert_entities_to_dicts(df):
    """Convert the entities dataframe to a list of dicts for NetworkX."""
    nodes_dict = {}
    for _, row in df.iterrows():
        # Create a dictionary for each row and collect unique nodes
        node_id = row["title"]
        if node_id not in nodes_dict:
            filtered_row = row[selected_node_columns]
            nodes_dict[node_id] = filtered_row.to_dict()
    return nodes_dict

# converts the relationships dataframe to a list of dicts for NetworkX
def convert_relationships_to_dicts(df):
    """Convert the relationships dataframe to a list of dicts for NetworkX."""
    relationships = []
    for _, row in df.iterrows():
        # Create a dictionary for each row
        filtered_row = row[selected_edge_columns]
        relationships.append((row["source"], row["target"], filtered_row.to_dict()))
    return relationships

### Run local search on sample queries

In [14]:
"""
Helper function to visualize the result context with NetworkX.

The dataframes are converted into supported nodes and relationships lists and then passed to NetworkX.
Additionally, some values are mapped to visualization properties.
"""

def show_graph(result, save_path):
    """Visualize the result context with NetworkX."""

    # converts the entities dataframe to a list of dicts for NetworkX
    def convert_entities_to_dicts(df):
        """Convert the entities dataframe to a list of dicts for NetworkX."""
        nodes_dict = {}
        for _, row in df.iterrows():
            # Create a dictionary for each row and collect unique nodes
            node_id = row["entity"]
            if node_id not in nodes_dict:
                nodes_dict[node_id] = row.to_dict()
        return nodes_dict

    # Create a NetworkX graph
    G = nx.DiGraph()

    # Add nodes and edges to the graph
    nodes = convert_entities_to_dicts(result.context_data["entities"])
    edges = convert_relationships_to_dicts(result.context_data["relationships"])

    for node_id, properties in nodes.items():
        G.add_node(node_id, **properties)

    for start, end, properties in edges:
        G.add_edge(start, end, **properties)

    # Draw the graph
    pos = nx.circular_layout(G)
    nx.draw(G, pos, with_labels=False, labels={node: node for node in G.nodes()}, node_size=4000, font_size=10, font_color="black", font_weight="bold", edge_color="gray", width=10)
    for node,(x,y) in pos.items():
        G.nodes[node]['x'] = float(x)
        G.nodes[node]['y'] = float(y)

    nx.write_graphml(G, save_path)

In [15]:
async def get_answer(df, type):
    answers = []
    for query in df['Query']:
        if not pd.isna(query):
            print(query)
            answer = await search_engine.search(query)
            answers.append(answer.response)

    df['answer'] = answers
    directory = f'./answers/{MODEL}/'
    os.makedirs(directory, exist_ok=True)
    df.to_excel(f"{directory}{MODEL} {type} local output.xlsx", index=False)

for type in ["Text", "TF", "Numerical", "Exp Design"]:
    df = pd.read_excel('../Query.xlsx', sheet_name=type, header=0)
    await get_answer(df, type)


What kind of hydrolases were reported in transcriptome analysis to be upregulated during the biodegradation of LDPE by Tenebrio molitor larvae? Please give answers in one short paragraph.
What specific compounds were reported in GC-MS analysis as degradation products of aged PE in the frass of mealworm larvae? Please list all of them in one short paragraph.
What were the main changes of functional groups of PE mulching films treated by coculture of Acinetobacter sp. strain NyZ450 and Bacillus sp. strain NyZ451 as reported? Please list all changes in term of peaks at the FTIR spectra in one short paragraph.
What bacterial genera were reported to be significantly upregulated in PE + bran diet group, compared with the sole bran diet group? Please only consider the mealworm gut microbiome studies for LDPE foam biodegradation and give answers in one short paragraph.
What bacterial genera were reported to be predominant (top 3 average relative abundance in the microbiome studies) in the gut 

In [ ]:
query = "What bacterial genera were reported to be significantly upregulated in PE + bran diet group, compared with the sole bran diet group? Please consider Lactobacillus and Mucispirillum."
result = await search_engine.asearch(query)

file_name = "3"
show_graph(result, f"./graphs/{file_name}.graphml")

In [ ]:
result